In [23]:
import os
from langgraph.graph import START, END, StateGraph
from typing_extensions import TypedDict, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image

load_dotenv(".env")
openai_client = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    api_key=os.getenv("OPENAI_API_TOKEN"),
    base_url=os.getenv("OPENAI_API_BASE"),
)


class JokeState(TypedDict):
    joke_name: str
    generated_joke: str
    joke_explation: str


def generate_joke(joke_state: JokeState) -> dict:
    prompt = f"Generate a funny joke about {joke_state['joke_name']}."
    response = openai_client.invoke(prompt)

    return {
        "generated_joke": response.content,
    }


def joke_explation(joke_state: JokeState) -> dict:
    prompt = f"explain the joke {joke_state['generated_joke']} in detail, where the joke name on the topic is {joke_state['joke_name']}."
    response = openai_client.invoke(prompt)
    return {"joke_explation": response.content}


joke_graph_check_memeory = MemorySaver()
joke_graph_builder = StateGraph(state_schema=JokeState)
joke_graph_builder.add_node("generate_joke", generate_joke)
joke_graph_builder.add_node("joke_explation", joke_explation)
joke_graph_builder.add_edge(START, "generate_joke")
joke_graph_builder.add_edge("generate_joke", "joke_explation")
joke_graph_builder.add_edge("joke_explation", END)
joke_graph = joke_graph_builder.compile(checkpointer=joke_graph_check_memeory)
try:
    Image(joke_graph.get_graph().draw_mermaid_png())
except Exception as e:
    print("Mermaid PNG rendering not supported in this environment.")

In [24]:
thread_id = 1
config = {"configurable": {"thread_id": thread_id}}
initial_state = {
    "joke_name": ["quantum mechanics"],
}
result = joke_graph.invoke(initial_state, config=config)
result

{'joke_name': ['quantum mechanics'],
 'generated_joke': "Why don't quantum physicists ever play hide and seek?\n\nBecause when they find you, they still don't know exactly where you are or how fast you're going!",
 'joke_explation': 'The joke "Why don\'t quantum physicists ever play hide and seek? Because when they find you, they still don\'t know exactly where you are or how fast you\'re going!" is a play on the principles of quantum mechanics, specifically the Heisenberg Uncertainty Principle.\n\n### Breakdown of the Joke:\n\n1. **Quantum Mechanics Context**:\n   - Quantum mechanics is a fundamental theory in physics that describes nature at the smallest scales, such as that of subatomic particles.\n   - It often defies classical intuition, leading to phenomena that seem strange or paradoxical.\n\n2. **Heisenberg Uncertainty Principle**:\n   - Proposed by Werner Heisenberg, this principle states that there is a fundamental limit to the precision with which certain pairs of physical p

In [25]:
joke_graph.get_state(config=config).values

{'joke_name': ['quantum mechanics'],
 'generated_joke': "Why don't quantum physicists ever play hide and seek?\n\nBecause when they find you, they still don't know exactly where you are or how fast you're going!",
 'joke_explation': 'The joke "Why don\'t quantum physicists ever play hide and seek? Because when they find you, they still don\'t know exactly where you are or how fast you\'re going!" is a play on the principles of quantum mechanics, specifically the Heisenberg Uncertainty Principle.\n\n### Breakdown of the Joke:\n\n1. **Quantum Mechanics Context**:\n   - Quantum mechanics is a fundamental theory in physics that describes nature at the smallest scales, such as that of subatomic particles.\n   - It often defies classical intuition, leading to phenomena that seem strange or paradoxical.\n\n2. **Heisenberg Uncertainty Principle**:\n   - Proposed by Werner Heisenberg, this principle states that there is a fundamental limit to the precision with which certain pairs of physical p

In [28]:
for state in list(joke_graph.get_state_history(config=config)):
    print(state.values)

{'joke_name': ['quantum mechanics'], 'generated_joke': "Why don't quantum physicists ever play hide and seek?\n\nBecause when they find you, they still don't know exactly where you are or how fast you're going!", 'joke_explation': 'The joke "Why don\'t quantum physicists ever play hide and seek? Because when they find you, they still don\'t know exactly where you are or how fast you\'re going!" is a play on the principles of quantum mechanics, specifically the Heisenberg Uncertainty Principle.\n\n### Breakdown of the Joke:\n\n1. **Quantum Mechanics Context**:\n   - Quantum mechanics is a fundamental theory in physics that describes nature at the smallest scales, such as that of subatomic particles.\n   - It often defies classical intuition, leading to phenomena that seem strange or paradoxical.\n\n2. **Heisenberg Uncertainty Principle**:\n   - Proposed by Werner Heisenberg, this principle states that there is a fundamental limit to the precision with which certain pairs of physical pro

##### Advantage of the persistance in langgraph
- short term memory
- fault tolerance
- HITL( human in the loop)
- Time travel